<a href="https://colab.research.google.com/github/11235813211/CATHRAMN-Test-Repository/blob/main/NGC_1501_SExtractor_for_Light_Curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Authorize Google Drive and Install Modules
!pip install astroquery
!pip install astropy
!pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import astroquery.gaia
import astropy.units as u
import astropy.coordinates
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import os
import re
import math
import bokeh.plotting as bk
import matplotlib as mpl
from bokeh.models import Range1d, Label
from bokeh.models import Title
from bokeh.layouts import row, gridplot
from scipy import stats
import numpy as np
import json
import requests

from IPython.display import Javascript

from bokeh.core.properties import Instance, String
from bokeh.io import show
from bokeh.models import ColumnDataSource, LayoutDOM
from bokeh.util.compiler import TypeScript


from astropy.io import fits
from astropy.wcs import WCS
from astropy.io.fits import PrimaryHDU, getdata, getheader
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, ZScaleInterval, ImageNormalize
from astropy.visualization.stretch import SquaredStretch
!pip install sep  # This is Source Extractor Photometry, or SExtractor, or sex  
import numpy as np
import sep
!pip install barycorrpy
from astropy.time import Time
from barycorrpy import utc_tdb

Mounted at /content/drive


In [ ]:
#@title Things that you install for plotting
!pip install matplotlib
!pip install bokeh
import bokeh.plotting as bk
import numpy as np
from bokeh.models import Range1d
from bokeh.layouts import row

In [ ]:
#@title Base 10 to Base 16 "hexify"
#function that converts to base 16--for colors
hexnums = ["0","1",'2','3','4','5','6','7','8','9','A','B','C','D','E','F']
def hexify(num):
  hexnum = ""
  addable = int(num/16)
  hexnum += hexnums[addable]
  num = num - 16*addable
  addable = int(num/1)
  hexnum += hexnums[addable]
  return hexnum

In [ ]:
#@title Plot Function
def make_plot (title, x_label, y_label):

  plot = bk.figure(
    tools = "pan,box_zoom,reset,save",
    title = title,
    x_axis_label = x_label, y_axis_label = y_label, plot_width=350, plot_height=350, 
  )
  return(plot)

colors =[]
for i in range(num_files):
  color = int(i*255/num_files)
  color = hexify(color)
  color = "#"+color+"0000"
  colors.append(color)

In [ ]:
#@title Function to get Data from images
def find_target_in_images(p, coords, name,apradius):
  fluxes = []
  files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]  # get list of files in directory p
  files = files[1:] #Only using this as LCO's first image was bleh
  num_files = len(files)
  print(num_files)
  for i in range(num_files):
    if not re.search("\.fits$", files[i]):  # ignore if not a .fits file
      continue
    img_path = os.path.join(p, files[i])  
    infits = fits.open(img_path)

    img_info = dict(infits[0].header)
    filt, site = img_info["FILTER"], img_info["SITE"]

    wrld = WCS(infits[0].header) # get the plate solution from the image header ("WCS" = "World Coordinate System" = RA and Dec)
    py, px = wrld.world_to_array_index(coords)    # pixel coordinates from WCS coordinates  

    data = infits[0].data
    data = data.astype(float)
    bkg = sep.Background(data)
    bkg_image = bkg.back()
    bkg_rms = bkg.rms()
    data_sub = data - bkg   # Subtract the background  
    objects = sep.extract(data_sub, 1.5, err=bkg.globalrms) 

    flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 
                                         apradius, err=bkg.globalrms, gain=1.0)      

    min_dist, closest_x, closest_y, closest_flux = 1000, 0, 0, 0
    for j in range(len(objects)):
      x, y, flux = (objects['x'][j], objects['y'][j], objects['flux'][j])
      pdist = (x-px)**2 + (y-py)**2          
      if pdist < min_dist:
        min_dist, closest_x, closest_y, closest_flux = pdist, x, y, flux

    wpx, wpy = wrld.wcs_pix2world(closest_x, closest_y, 1)
    ra = float(wpx)
    dec = float(wpy)

    coords_in_image = SkyCoord(wpx, wpy, unit="deg")

    fluxes.append(round(closest_flux))
    if closest_flux == 0:
      print(f"{files[i]} ({filt} filter): {name} not found in image")
  return (num_files,fluxes)

In [ ]:
FLUXES = []

In [ ]:
# p stands for "path".  Use the folder icon at the left to navigate to the folder of images.
# Then click the "Copy Path" button and copy-paste the path into the "p" variable below.

apradius = 7.0
p = '/content/drive/MyDrive/C A T H R A M N/Images/NGC 1501'
coords = SkyCoord("04h06m59.38s+60d55m14.6s")
name = "CH Cam"

(num_files,fluxes) = find_target_in_images(p, coords, name,apradius)
FLUXES.append(fluxes)
print(FLUXES)

11


Set OBSGEO-B to    20.706969 from OBSGEO-[XYZ].
Set OBSGEO-H to     3033.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


[[115933, 116896, 115692, 105449, 110675, 103663, 104709, 105517, 108667, 108414, 105406]]


In [ ]:
#1111111111111111111111111111111111111111
#COMP 1
coords = SkyCoord("04h07m31.35s+60d57m38.8s")
name = "CH Cam COMP #1"

(num_files,fluxescomp1) = find_target_in_images(p, coords, name, apradius)
FLUXES.append(fluxescomp1)

11


Set OBSGEO-B to    20.706969 from OBSGEO-[XYZ].
Set OBSGEO-H to     3033.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [ ]:
#2222222222222222222222222222222222222222
#COMP 2
coords = SkyCoord("04h07m07.12s+60d59m48.3s")
name = "CH Cam COMP #2"

(num_files,fluxescomp2) = find_target_in_images(p, coords, name, apradius)
FLUXES.append(fluxescomp2)

11


Set OBSGEO-B to    20.706969 from OBSGEO-[XYZ].
Set OBSGEO-H to     3033.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [ ]:
#3333333333333333333333333333333333333333
#COMP 3
coords = SkyCoord("04h07m39.90s+60d57m48.7s")
name = "CH Cam COMP #3"

(num_files,fluxescomp3) = find_target_in_images(p, coords, name, apradius)
FLUXES.append(fluxescomp3)

11


Set OBSGEO-B to    20.706969 from OBSGEO-[XYZ].
Set OBSGEO-H to     3033.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [ ]:
#Useful Stuff
image_time = float(input("Enter the time of the images taken in minutes: "))
compmags = input("Entre the magnitudes of the comps, in order, as follows a,b,c (NO SPACES!!): ")
compmags = compmags.split(",")
for i in range(len(compmags)):
  compmags[i] = float(compmags[i])
print(compmags)

Enter the time of the images taken in minutes: 2.5
Entre the magnitudes of the comps, in order, as follows a,b,c (NO SPACES!!): 14.081,14.314,14.758
[14.081, 14.314, 14.758]


In [ ]:
x = []
for i in range(num_files):
  x.append(i*image_time)

In [ ]:
#THIS DOESN'T COMPARE THE COMPS AND TARGET--PLOTS INDIVIDUAL STARS (YEAH, IT'S NOT NECESSARY)
bk.output_notebook()

plots = [] #ADJUST FOR THE NUMBER OF COMPS!

for i in range(len(FLUXES)):
  flux = FLUXES[i]
  if i == 0:
    plotnamey = "CH Cam"
  else:
    plotnamey = f"Comp Star #{i}"
  plotty = make_plot(plotnamey,"Time (minutes)","Flux (counts)")
  plotty.scatter(x,flux,color=colors)
  plots.append(plotty)


bk.show(row(plots))

In [ ]:
bk.output_notebook()

plots = []
rel_flux = []

for i in range(len(FLUXES)-1):
  fluxcomp = FLUXES[i+1]
  rel_flux.append([])
  for v in range(num_files):
    rel_mag = FLUXES[0][v]/FLUXES[i+1][v]
    rel_mag = compmags[i] - 2.501*math.log10(rel_mag)
    rel_flux[i].append(rel_mag)

for i in range(len(rel_flux)):
  flux = rel_flux[i]
  plotnamey = f"CH Cam: Comp Star {i+1}"
  plotty = make_plot(plotnamey,"Time (minutes)","Differential Magnitude")
  plotty.scatter(x,flux,color=colors)
  plots.append(plotty)


bk.show(row(plots))